In [9]:
import numpy as np
import matplotlib.pyplot as plt
import subprocess
from configobj import ConfigObj
from itertools import chain
import os, glob
import shutil
import pandas as pd

In [2]:
#define paramters to be modified in default config file
#for any section that requires paramters to be edited, include it in 'sed_modules_params'
CIGALE_PARAMS = {'data_file': 'sdss_qso.txt',
                 'sed_modules': ['sfhdelayed', 'bc03', 'nebular', 'dustatt_calzleit', 'dale2014', 'skirtor2016', 'yang20', 'redshifting'],
                 'analysis_method': "pdf_analysis",
                 'sed_modules_params': {
                     'sfhdelayed': {
                        'tau_main': '500',
                        'age_main': ['100', '3e3', '5e3'],
                     },
                     'skirtor2016': {
                         'delta': -0.36,
                         'fracAGN': 0.9999999,
                         'EBV': ['0.', '0.05', '0.1', '0.15', '0.2', '0.3'],
                     }
                 },
                 'analysis_params': {
                     'save_best_sed': True,
                 }}

In [3]:
def prep_cigale(configfile, params):

    subprocess.run(["pcigale", "init"])

    #read in configfile
    config = ConfigObj(configfile, encoding = 'utf-8')
    config.filename = configfile

    #edit params
    config['data_file'] = params['data_file']
    config['sed_modules'] = params['sed_modules']
    config['analysis_method'] = params['analysis_method']
    config.write()
    
    #generate config file to run cigale
    try:
        subprocess.run(["pcigale", "genconf"])
    except:
        raise ValueError("Could not create config file - check defaults")
    
    #read configfile again as it has been modified by genconf
    config = ConfigObj(configfile, encoding = 'utf-8')
    config.filename = configfile
    
    #genconf will overwrite any edits, so modify paramters at the end
    for module_ in params['sed_modules']:
        if module_ in params['sed_modules_params'].keys():
            mod = params['sed_modules_params'][module_]
            for key, value in mod.items():
                config['sed_modules_params'][module_][key] = value

    if 'analysis_params' in params.keys():
        for key, value in params['analysis_params'].items():
            config['analysis_params'][key] = value
    config.write()

    #verify if config file is ok
    try:
        subprocess.run(["pcigale", "check"])
    except:
        raise ValueError("Config file in incorrect")
    
    return config

In [4]:
def run_cigale(configfile, params, outdir, plot=False):
    if not os.path.exists(outdir):
        os.makedirs(outdir)

    #copy everything to outdir
    shutil.copy(configfile, os.path.join(outdir, configfile))
    shutil.copy(configfile + '.spec', os.path.join(outdir, configfile + '.spec'))
    shutil.copy(params['data_file'], os.path.join(outdir, params['data_file']))

    cwd = os.getcwd()
    os.chdir(outdir)

    #run cigale
    subprocess.run(["pcigale", "run"])
    if plot:
        subprocess.run(["pcigale-plots", "sed"])
    os.chdir(cwd)

In [34]:
def read_ptf_sample(filename):
    try:
        ptf_sne = pd.read_csv(filename, header = None, names=['SN', 'telescope', 'instrument', 'filter', 'mag', 'magerr', 'ref'])
        ptf_sne['SN'] = ptf_sne['SN'].str.strip()
        ptf_sne['telescope'] = ptf_sne['telescope'].str.strip()
        ptf_sne['instrument'] = ptf_sne['instrument'].str.strip()
        ptf_sne['filter'] = ptf_sne['filter'].str.strip()
        
        return ptf_sne
    except:
        raise ValueError('Reformat input file with correct whitespaces and delimiters')

In [5]:
config = prep_cigale('pcigale.ini', CIGALE_PARAMS)

╭──────────────────────────────────────────────────────────────────────────────╮
│                      Code Investigating GALaxy Emission                      │
│                Boquien et al. (2019) (]8;id=445196;https://cigale.lam.fr\https://cigale.lam.fr]8;;\)                 │
│         CIGALE version: 2025.0 — Python version: 3.12.0 — Platform:          │
│                              macosx-11.1-arm64                               │
╰───────────────────────────���──────────────────────────────────────────────────╯


                                  SED modules                                   
╭──────────────────────┬───────────────────────────────────────────────────────╮
│ SFH                  │ None. Options are: sfh2exp, sfh_buat08, sfhdelayed,   │
│                      │ sfhdelayedbq, sfhfromfile, sfhperiodic.               │
│ SSP                  │ None. Options are: bc03, bpassv2, cb19, m2005.        │
│ nebular              │ None. Options are: nebular.         

In [6]:
run_cigale('pcigale.ini', CIGALE_PARAMS, 'sdss', plot=True)

╭──────────────────────────────────────────────────────────────────────────────╮
│                      Code Investigating GALaxy Emission                      │
│                Boquien et al. (2019) (]8;id=600744;https://cigale.lam.fr\https://cigale.lam.fr]8;;\)                 │
│         CIGALE version: 2025.0 — Python version: 3.12.0 — Platform:          │
│                              macosx-11.1-arm64                               │
╰───────────────────────────���──────────────────────────────────────────────────╯


                              General information                               
                ╭───────────────────┬─────────────────────────╮                 
                │ Data file         │ sdss_qso.txt            │                 
                │ Parameters file   │ None                    │                 
                │ Number of objects │ 11                      │                 
                │ Redshift          │ 0.78 to 2.86           

In [40]:
ptf_sne = read_ptf_sample('ptf_ccsne.csv')

In [42]:
ptf_sne

,SN,telescope,instrument,filter,mag,magerr,ref
0,PTF09atu,VLT,FORS2,g,27.04,0.30,[2018MNRAS.473.1258S]
1,PTF09atu,VLT,FORS2,z,25.69,0.46,[2018MNRAS.473.1258S]
2,PTF09atu,VLT,FORS2,R,26.20,0.24,[2018MNRAS.473.1258S]
3,PTF09atu,VLT,FORS2,I,26.05,0.40,[2018MNRAS.473.1258S]
4,PTF09atu,VLT,HAWK-I,J,26.61,1.12,[2018MNRAS.473.1258S]
...,...,...,...,...,...,...,...
11128,iPTF17zg,2MASS,,J,13.77,0.07,This paper
11129,iPTF17zg,2MASS,,H,13.41,0.07,This paper
11130,iPTF17zg,2MASS,,Ks,13.61,0.11,This paper
11131,iPTF17zg,WISE,,W1,14.10,0.10,This paper
